# Multiscattering Demo v2 (Jetson-friendly)
Minimal example using quantem-style APIs and synthetic data.


In [1]:
import sys
from pathlib import Path

# Add parent dir to path (for running without pip install)
root = Path.cwd().parent
if str(root) not in sys.path:
    sys.path.insert(0, str(root))

import torch
from src.config import CONFIG_QUICK
from src.data_generation import generate_synthetic_dataset
from src.reconstruction import ReconstructionEngine
from src.forward_models import create_single_slice_model
from src.metrics import compute_amplitude_rmse, compute_phase_mae

config = CONFIG_QUICK
print(config)

ExperimentConfig(object_size=96, probe_size=32, diffraction_size=96, scan_grid=24, pixel_size=0.05, voltage_kev=200.0, convergence_angle=0.02, slice_thickness=0.5, num_iterations=120, learning_rate=0.2, batch_size=32)


In [2]:

# Generate a tiny synthetic dataset
syn = generate_synthetic_dataset(config)
patterns = syn.diffraction_patterns
positions = syn.scan_positions
print('patterns', patterns.shape)


patterns torch.Size([576, 96, 96])


In [3]:

# Single-slice reconstruction (lightweight)
model = create_single_slice_model(config, positions.to(config.device))
engine = ReconstructionEngine(model, lr_object=0.2, lr_probe=0.05)
result = engine.reconstruct(patterns.to(config.device), num_iterations=60, batch_size=32, min_iterations=20, convergence_tol=1e-4, verbose=False)
print('final loss', result.final_loss)


final loss 1.4687147818727277e-05


In [4]:

# Quick metrics vs ground truth
recon = result.reconstruction
gt = syn.ground_truth
print('amp RMSE', compute_amplitude_rmse(recon, gt))
print('phase MAE', compute_phase_mae(recon, gt))


amp RMSE 0.38812708854675293
phase MAE 0.06850888580083847
